# All-or-nothing

We have 62 % chance to predict if the price will go up or down.

```
X = df_norm[[OPEN, HIGH, LOW, CLOSE, VOLUME]]
y = df_norm['aon']
```

`aon` column is define like this: 1 if price have increase from the previous row, -1 otherwise.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn import tree, svm, linear_model, neural_network, gaussian_process
from sklearn.model_selection import train_test_split, cross_val_score, TimeSeriesSplit, GridSearchCV, ParameterGrid, cross_validate
from sklearn import preprocessing
from sklearn import metrics
import pandas as pd
import numpy as np
import datetime
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

'en_US.UTF-8'

In [3]:
TIME = 'time'
OPEN = 'open'
HIGH = 'high'
LOW = 'low'
CLOSE = 'close'
VWAP = 'vwap'
VOLUME = 'volume'
COUNT = 'count'

In [4]:
# Open the dataset
df = pd.read_csv("ohlc-btc-eur-day.csv")

In [5]:
# Select columns
df_num = df[[OPEN, HIGH, LOW, CLOSE, VOLUME]]
df_num.shape

(720, 5)

In [6]:
# Scale data
scaler = preprocessing.StandardScaler()
df_norm = pd.DataFrame(scaler.fit_transform(df_num.values), columns=df_num.columns, index=df_num.index)

# Unscale data
def unscale(value, key=CLOSE):
    return value * np.std(df_num[key]) + np.mean(df_num[key])

In [7]:
# Prepare

aon = [0]
for index, row in df_norm.iterrows():
    if index == df_norm.shape[0] - 1:
        continue
    aon.append(1 if row[CLOSE] < df_norm[CLOSE][index+1] else -1)

df_norm.insert(df_norm.shape[1], 'aon', aon)

In [10]:
# Train model

X = df_norm[[OPEN, HIGH, LOW, CLOSE, VOLUME]]
y = df_norm['aon']

X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel())
model = neural_network.MLPRegressor(hidden_layer_sizes=(100,),
                                    activation='relu',
                                    solver='lbfgs',
                                    alpha=1
                                   )

model.fit(X_train, y_train)

MLPRegressor(alpha=1, solver='lbfgs')

In [12]:
# Evaluate the model

def aon(y_pred, y_true):
    # all-or-none principle
    def func(val):
        return 1 if val > 0 else -1
    return func(y_pred) == y_true

def aon_scorer(estimator, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    estimator.fit(X_train, y_train)
    y_pred = estimator.predict(X_test).reshape(X_test.shape[0], 1)
    X_test_close = pd.DataFrame(X_test[CLOSE])
    X_test_close.insert(1, 'y_test', y_test)
    X_test_close.insert(1, 'y_pred', y_pred)
    aon_series = X_test_close.apply(lambda val: aon(val['y_pred'], val['y_test']), axis=1)
    return np.mean(aon_series)

cross = cross_val_score(model, X, y, scoring=aon_scorer)
print(np.mean(cross))
cross

0.47777777777777775


array([0.52777778, 0.44444444, 0.36111111, 0.5       , 0.55555556])